In [3]:
#importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


dataset = pd.read_csv('train.csv')

In [4]:
dataset['Timestep'] = dataset.Timestep.str[2:6]
dataset['Timestep'].astype(int)


0        1810
1        1810
2        1810
3        1810
4        1810
         ... 
89995    1986
89996    1986
89997    1986
89998    1986
89999    1986
Name: Timestep, Length: 90000, dtype: int64

In [5]:
one_hot_encoded_data = pd.get_dummies(dataset, columns = ['Course', 'Faculty'])
dataset = one_hot_encoded_data
dataset.insert(0, 'Expected', dataset.pop('Expected'))
X = dataset.iloc[:,1:6]
Y = dataset.iloc[:,0]

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 0)

In [5]:
# Importing standardscalar module 
from sklearn.preprocessing import StandardScaler
  
scalar = StandardScaler()
  
# fitting
scalar.fit(X_train)
scaled_data = scalar.transform(X_train)
  
# Importing PCA
from sklearn.decomposition import PCA
  
# Let's say, components = 2
pca = PCA(n_components = 5)
pca.fit(X_train)
x_pca = pca.transform(X_train)
  
x_pca.shape


(72000, 5)

In [6]:
# Importing standardscalar module 
from sklearn.preprocessing import StandardScaler
  
scalar = StandardScaler()
  
# fitting
scalar.fit(X_test)
scaled_data = scalar.transform(X_test)
  
# Importing PCA
from sklearn.decomposition import PCA
  
# Let's say, components = 2
pca = PCA(n_components = 5)
pca.fit(X_test)
xt_pca = pca.transform(X_test)
  
xt_pca.shape


(18000, 5)

Linear Regression

In [7]:
from sklearn import linear_model

regr = linear_model.LinearRegression()
regr.fit(X_train, Y_train)

y_pred = regr.predict(X_test)

KNN

In [8]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(X_train, Y_train)

KNeighborsClassifier(n_neighbors=3)

Random Forest Regressor

In [9]:
y_pred = pd.Series(neigh.predict(X_test), index = X_test.index)

In [17]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
X_train, Y_train = make_regression(n_features=5, n_informative=2,random_state=0, shuffle=False)
regr = RandomForestRegressor(max_depth=2, random_state=0)
regr.fit(X_train, Y_train)
y_pred = regr.predict(X_test)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


SGD(best model)

In [19]:
from sklearn.linear_model import SGDRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
n_samples, n_features = 90000, 5
#rng = np.random.RandomState(0)
#Y_train = rng.randn(n_samples)
#X_train = rng.randn(n_samples, n_features)
# Always scale the input. The most convenient way is to use a pipeline.
reg = make_pipeline(StandardScaler(),SGDRegressor(loss='squared_error', penalty='l2', alpha=0.0001, l1_ratio=0.15, fit_intercept=True, max_iter=1000, tol=0.001, shuffle=True, verbose=0, epsilon=0.1, random_state=None, learning_rate='invscaling', eta0=0.01, power_t=0.25, early_stopping=False, validation_fraction=0.1, n_iter_no_change=5, warm_start=False, average=False))
reg.fit(X_train, Y_train)
y_pred = reg.predict(X_test)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


ABove is the best trained model


In [20]:
from sklearn.metrics import mean_squared_error
mean_squared_error(Y_test, y_pred, squared=False)

2549096.6656355713

In [ ]:
df = pd.read_csv('test.csv')

In [ ]:
df['Timestep'] = df.Timestep.str[2:6]
df['Timestep'].astype(int)

In [ ]:
one_hot_encoded_data = pd.get_dummies(df, columns = ['Course', 'Faculty'])
df = one_hot_encoded_data
df

In [ ]:
X_f = df.iloc[:,0:1219]
# Importing standardscalar module 
from sklearn.preprocessing import StandardScaler
  
scalar = StandardScaler()
  
# fitting
scalar.fit(X_f)
scaled_data = scalar.transform(X_f)
  
# Importing PCA
from sklearn.decomposition import PCA
  
# Let's say, components = 2
pca = PCA(n_components = 5)
pca.fit(X_f)
x_pca = pca.transform(X_f)
  
x_pca.shape
y_pred_f = pd.Series(regr.predict(x_pca))
y_pred_f

In [ ]:
df1 = pd.DataFrame(data = y_pred_f.index, columns = ['Id'])
df2 = pd.DataFrame(data = y_pred_f.values, columns = ['Predicted'])
df3 = pd.merge(df1, df2, left_index = True, right_index = True)
df3.to_csv('submission.csv', index = False)